In [18]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
import folium

In [2]:
# Define bounding box from the coordinates you found
min_lat, max_lat = 51.524, 51.5405
min_lon, max_lon = -0.0745, -0.0317

# Cell size in degrees (~200m ~ 0.0018 degrees)
cell_size = 0.0018

# Create grid
lat_steps = np.arange(min_lat, max_lat, cell_size)
lon_steps = np.arange(min_lon, max_lon, cell_size)

grid_cells = []
cell_id = 0
for i, lat in enumerate(lat_steps):
    for j, lon in enumerate(lon_steps):
        grid_cells.append({
            'cell_id': f'cell_{i}_{j}',
            'geometry': Polygon([
                (lon, lat),
                (lon + cell_size, lat),
                (lon + cell_size, lat + cell_size),
                (lon, lat + cell_size)
            ])
        })
        cell_id += 1

gdf = gpd.GeoDataFrame(grid_cells)
gdf.to_file("london_grid.geojson", driver="GeoJSON")
print("Grid created:", len(grid_cells), "cells")

Grid created: 240 cells


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [5]:
band4_file = "Sentinel-2_L2A_B04_(Raw).tiff"
band8_file = "Sentinel-2_L2A_B08_(Raw).tiff"

# This is the grid file you created in your previous step.
# Make sure this path is correct!
grid_file = "london_grid.geojson" 
print("Loading raster files...")

Loading raster files...


In [6]:
# --- 1. Load the bands ---
with rasterio.open(band4_file) as src:
    red = src.read(1).astype('float32')
    profile = src.profile  # Get the metadata (projection, size, etc.)

with rasterio.open(band8_file) as src:
    nir = src.read(1).astype('float32')

print("Calculating NDVI...")

Calculating NDVI...


In [7]:

# --- 2. Calculate NDVI ---
# Formula: (NIR - Red) / (NIR + Red)
# We must handle cases where (NIR + Red) is zero to avoid errors

# Set warnings to ignore division by zero (we will handle it)
np.seterr(divide='ignore', invalid='ignore')

# Calculate the denominator
denominator = nir + red

# Create the NDVI array
# Where denominator is 0, set NDVI to 0, otherwise calculate it
ndvi = np.where(denominator == 0, 0, (nir - red) / denominator)

# Update the metadata for our new NDVI file
profile.update(
    dtype='float32',
    count=1,
    driver='GTiff'
)

In [8]:
# --- 3. Save the NDVI raster ---
ndvi_file = "ndvi.tif"
with rasterio.open(ndvi_file, 'w', **profile) as dst:
    dst.write(ndvi, 1)

print(f"NDVI calculation complete. Saved to {ndvi_file}")

# --- 4. Extract NDVI values for your grid ---
print(f"Extracting NDVI statistics for {grid_file}...")
# This function calculates the 'mean' NDVI for each polygon in your grid
# It returns a list of dictionaries, e.g., [{'mean': 0.25}, {'mean': 0.31}, ...]
stats = zonal_stats(grid_file, ndvi_file, stats="mean")


NDVI calculation complete. Saved to ndvi.tif
Extracting NDVI statistics for london_grid.geojson...


In [9]:
# --- 5. Combine data and save the final file ---
print("Adding NDVI stats to GeoDataFrame...")
# Read your original grid
gdf = gpd.read_file(grid_file)

# Get just the 'mean' value from each dictionary in the 'stats' list
mean_ndvi = [s['mean'] for s in stats]

# Add this list as a new column to your GeoDataFrame
gdf['mean_ndvi'] = mean_ndvi

# Save the final, combined file
final_file = "london_grid_with_ndvi.geojson"
gdf.to_file(final_file, driver="GeoJSON")

print("---")
print(f"🎉 Success! Final file saved to {final_file}")
print("This file contains your grid with a 'mean_ndvi' column for each cell.")

Adding NDVI stats to GeoDataFrame...
---
🎉 Success! Final file saved to london_grid_with_ndvi.geojson
This file contains your grid with a 'mean_ndvi' column for each cell.


In [10]:
# --- 1. DEFINE FILES ---
# This is the output from Task 2
ndvi_grid_file = "london_grid_with_ndvi.geojson"

# This is the temperature file you downloaded in Task 3
temp_file = "Landsat_8-9_L2_B10_(Raw).tiff"

In [11]:
# --- 2. EXTRACT TEMPERATURE VALUES ---
print("Extracting temperature statistics...")
# Use zonal_stats again, this time on the temperature file
temp_stats = zonal_stats(ndvi_grid_file, temp_file, stats="mean")

# --- 3. LOAD YOUR NDVI GRID ---
gdf = gpd.read_file(ndvi_grid_file)
print("Loaded NDVI grid.")


Extracting temperature statistics...
Loaded NDVI grid.


In [12]:

# --- 4. CONVERT TEMP & CALCULATE HEAT SCORE ---
print("Calculating heat scores...")
heat_scores = []
for i, row in gdf.iterrows():
    # Get the raw mean value for the cell
    raw_temp = temp_stats[i]['mean']
    
    if raw_temp is not None:
        # CONVERT TO CELSIUS (This is the accurate way)
        # Landsat L2 data is scaled. Formula: (DN * 0.00341802) + 149.0 = Kelvin
        temp_kelvin = (raw_temp * 0.00341802) + 149.0
        
        # Convert Kelvin to Celsius
        temp_celsius = temp_kelvin - 273.15
        
        # Get the NDVI value we already calculated
        mean_ndvi = row['mean_ndvi']
        
        # Apply your formula:
        heat_score = temp_celsius - (0.3 * mean_ndvi)
        heat_scores.append(heat_score)
        
    else:
        heat_scores.append(None) # In case a cell had no data

# Add the final score as a new column
gdf['heat_score'] = heat_scores


Calculating heat scores...


In [14]:

# --- 5. SAVE FINAL FILE ---
final_output_file = "london_grid_with_heat_score.geojson"
gdf.to_file(final_output_file, driver="GeoJSON")

print("---")
print(f"🎉 ALL TASKS COMPLETE! Final file saved to {final_output_file}")

---
🎉 ALL TASKS COMPLETE! Final file saved to london_grid_with_heat_score.geojson


In [19]:
# --- 1. DEFINE YOUR FINAL FILE ---
final_file = "london_grid_with_heat_score.geojson"

print(f"Loading final file: {final_file}")
gdf = gpd.read_file(final_file)

Loading final file: london_grid_with_heat_score.geojson


In [20]:
# --- 2. FIND THE CENTER OF YOUR MAP ---
# We get the center of your grid to tell Folium where to look
center_y = (gdf.bounds.miny.min() + gdf.bounds.maxy.max()) / 2
center_x = (gdf.bounds.minx.min() + gdf.bounds.maxx.max()) / 2
map_center = [center_y, center_x]

In [24]:
print(map_center)

[np.float64(51.533), np.float64(-0.0529)]


In [21]:
# --- 3. CREATE THE BASE MAP ---
# zoom_start=14 is a good zoom level for a small area
m = folium.Map(location=map_center, zoom_start=14, tiles="CartoDB positron")

In [22]:
# --- 4. CREATE THE HEAT SCORE LAYER ---
# This tells Folium to use your GeoJSON, find the 'heat_score'
# for each 'cell_id', and color it using a 'Yellow-Orange-Red' scale.
folium.Choropleth(
    geo_data=gdf,
    name='Heat Score',
    data=gdf,
    columns=['cell_id', 'heat_score'], # 'cell_id' is the key, 'heat_score' is the value
    key_on='feature.properties.cell_id', # This links the data to the map geometry
    fill_color='YlOrRd', # This is a great color scale for heat
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Heat Score (Higher = Hotter)',
    highlight=True # Lets you see values when you hover
).add_to(m)

# Add a layer control to turn the heat map on/off
folium.LayerControl().add_to(m)

In [23]:
# --- 5. SAVE THE MAP ---
map_file = "heat_score_map.html"
m.save(map_file)

print("---")
print(f"🎉 SUCCESS! Your map is saved to: {map_file}")
print("Just double-click that file to open it in your browser!")

---
🎉 SUCCESS! Your map is saved to: heat_score_map.html
Just double-click that file to open it in your browser!
